In [23]:
%load_ext autoreload
%autoreload 2
%matplotlib qt5
# Add modules to the path
import sys
sys.path.append('C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git\\')
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos')
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git\\freely_moving_photometry_analysis')
import matplotlib.pyplot as plt
from hernando_behavioural_analysis_dev.BehaviouralAnalysis.utils import custom_functions as cuf
import matplotlib.cm as cm
import numpy as np
import os
import matplotlib
import pandas as pd
import pickle
from utils.post_processing_utils import *
import seaborn as sns

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
def moving_average(x, window=50):
    return np.convolve(x, np.ones(window), 'valid') / window

In [54]:
trial_data

,Trial num,Trial type,State type,Instance in state,Max times in state,State name,Time start,Time end,Response,Last response,Last outcome,Trial start,Trial end,Trial outcome,First response,Reward block,First choice correct
0,0.0,7.0,1.0,1.0,1.0,TrialStart,26.2907,26.3007,2.0,-1.0,-1.0,26.2907,41.6852,1.0,2.0,0.0,1.0
1,0.0,7.0,2.0,1.0,3.0,WaitForPoke,26.3007,35.9298,2.0,-1.0,-1.0,26.2907,41.6852,1.0,2.0,0.0,1.0
2,0.0,7.0,3.0,1.0,3.0,CueDelay,35.9298,36.0016,2.0,-1.0,-1.0,26.2907,41.6852,1.0,2.0,0.0,1.0
3,0.0,7.0,2.0,2.0,3.0,WaitForPoke,36.0016,36.1435,2.0,-1.0,-1.0,26.2907,41.6852,1.0,2.0,0.0,1.0
4,0.0,7.0,3.0,2.0,3.0,CueDelay,36.1435,36.2206,2.0,-1.0,-1.0,26.2907,41.6852,1.0,2.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2892,267.0,1.0,3.0,1.0,1.0,CueDelay,1935.3221,1935.4221,2.0,1.0,1.0,1915.5516,1945.0576,0.0,2.0,1.0,0.0
2893,267.0,1.0,4.0,1.0,1.0,WaitForPortOut,1935.4221,1936.4146,2.0,1.0,1.0,1915.5516,1945.0576,0.0,2.0,1.0,0.0
2894,267.0,1.0,5.0,1.0,1.0,WaitForResponse,1936.4146,1943.0576,2.0,1.0,1.0,1915.5516,1945.0576,0.0,2.0,1.0,0.0
2895,267.0,1.0,9.0,1.0,1.0,Punish,1943.0576,1945.0576,2.0,1.0,1.0,1915.5516,1945.0576,0.0,2.0,1.0,0.0


In [56]:
processed_data_dir = os.path.join('W:\\photometry_2AC\\processed_data\\state_change_data')
state_change_data_file = os.path.join(processed_data_dir, 'value_switch_nacc_mice.csv')
pre_pc = []
post_pc = []
mice = [ 'SNL_photo28', 'SNL_photo30', 'SNL_photo31', 'SNL_photo32', 'SNL_photo34', 'SNL_photo35']
mouse_ids = []
session_ids = []
moving_avs = []
reward_blocks = []
#mice = ['SNL_photo28', 'SNL_photo30', 'SNL_photo31', 'SNL_photo32', 'SNL_photo33', 'SNL_photo34', 'SNL_photo35']
for mouse_num, mouse_id in enumerate(mice):
    sessions = all_experiments[(all_experiments['mouse_id'] == mouse_id) & (all_experiments['experiment_notes'] == 'value switch')]['date'].values
    for date in sessions:
        all_experiments = get_all_experimental_records()
        experiment_to_process = all_experiments[(all_experiments['date'] == date) & (all_experiments['mouse_id'] == mouse_id)]
        traces, trial_data = open_experiment(experiment_to_process)
        trial_data.loc[trial_data['Trial outcome'] == 3, 'Trial outcome'] = 0
        red_trial_data = trial_data[trial_data['State name'] == 'TrialStart']
        post_trials = red_trial_data[red_trial_data['Trial num'] >= 100]
        pre_trials = red_trial_data[red_trial_data['Trial num'] < 100]
        post_reward_side = post_trials['Reward block'].unique()[0]
        if post_reward_side == 1:
            side = -1
        elif post_reward_side == 5:
            side = 1

        pre_bias = cuf.RBias(pre_trials['First response'], pre_trials['First choice correct'])* side
        post_bias = cuf.RBias(post_trials['First response'], post_trials['First choice correct']) *side
        post_pc.append(post_bias)
        pre_pc.append(pre_bias)
        session_ids.append(date)
        mouse_ids.append(mouse_id)
        reward_blocks.append(post_reward_side)
        
        


In [73]:
behavioural_change = {}
behavioural_change['mouse'] = mouse_ids
behavioural_change['session'] = session_ids
behavioural_change['pre bias'] = pre_pc
behavioural_change['post bias'] = post_pc
behavioural_change_df = pd.DataFrame(behavioural_change)

In [76]:
df_for_plot

,mouse,pre bias,post bias
0,SNL_photo28,0.055000,0.088261
1,SNL_photo30,0.000000,0.078807
2,SNL_photo31,-0.016667,0.064614
3,SNL_photo32,-0.046667,0.023222
4,SNL_photo34,0.003333,0.061284
5,SNL_photo35,-0.023333,0.028219


In [86]:
df_for_plot = behavioural_change_df.groupby(['mouse'])[['pre bias', 'post bias']].apply(np.mean)
df_for_plot = df_for_plot.reset_index()

In [97]:
df_for_plot1 = df_for_plot.set_index('mouse').transpose()


In [102]:
def francescas_plot(ax, data, show_err_bar=False, mean_linewidth=4, mean_line_color='blue', legend=False):
    """Plot lines for all mice from early to late.
    """
    data.plot(ax=ax, color='gray', legend=False)
    data.mean(1).plot(ax=ax,linewidth=mean_linewidth,color=mean_line_color)

    if show_err_bar:
        yerr = data.std(axis=1)

        plt.errorbar(np.array([0,1]), data.mean(1), yerr, color=mean_line_color, linewidth=4)

    # drop the left and bottom spine for cool looking effect
    ax.spines['left'].set_position(('outward', 10))
    ax.spines['bottom'].set_position(('outward', 10))
    # Hide the right and top spines
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

In [103]:
fig, ax = plt.subplots(figsize=[3.5,4])
francescas_plot(ax, df_for_plot1, mean_line_color='#7FB5B5', mean_linewidth=6, show_err_bar=False)
plt.xticks([0, 1], ['equal rewards', 'unequal rewards'], fontsize=12)
plt.ylabel('Bias towards BIG side', fontsize=12)
plt.tight_layout()


In [77]:
df_for_plot

,mouse,pre bias,post bias
0,SNL_photo28,0.055000,0.088261
1,SNL_photo30,0.000000,0.078807
2,SNL_photo31,-0.016667,0.064614
3,SNL_photo32,-0.046667,0.023222
4,SNL_photo34,0.003333,0.061284
5,SNL_photo35,-0.023333,0.028219


In [418]:
processed_data_dir = os.path.join('W:\\photometry_2AC\\processed_data\\state_change_data')
nacc_peaks = pd.read_csv(os.path.join(processed_data_dir, 'nacc_peak_sizes.csv'))
tail_peaks = pd.read_csv(os.path.join(processed_data_dir, 'tail_peak_sizes.csv'))

In [419]:
peaks = nacc_peaks 

In [420]:
peak_differences = []
performance_differences = []
for i, row in behavioural_change_df.iterrows():
    mouse = row['mouse']
    peak_differences.append((peaks[mouse][1] - peaks[mouse][0])/peaks[mouse][1]*100)
    performance_differences.append((df_for_plot[mouse][1] - df_for_plot[mouse][0])/df_for_plot[mouse][1]*100)

In [422]:
fig, axs = plt.subplots()
axs.scatter(performance_differences, peak_differences)
axs.set_ylabel('% change in post-pre peak z-score ')
axs.set_xlabel('% difference in performance')


Text(0.5, 0, '% difference in performance')

In [174]:
(peaks[mouse][1] - peaks[mouse][0])

0.5066578094709958

In [175]:
(3-4)/5

-0.2

In [176]:
peaks

,trial type,SNL_photo21,SNL_photo22,SNL_photo26
0,pre,0.266602,0.896028,0.780225
1,post,0.574251,0.894707,1.286883
